# 데이터 로드

크롤링 한 데이터에서 수집기간에 맞지 않은 데이터들을 제외하고 하나의 데이터 프레임으로 병합

In [ ]:
import pandas as pd
import openpyxl

doc1 = pd.read_excel("c:\\naver_qna\\우울증_0101_0115_crawling_result.xlsx")
doc2 = pd.read_excel("c:\\naver_qna\\우울증_0116_0131_crawling_result.xlsx")
doc3 = pd.read_excel("c:\\naver_qna\\우울증_0201_0228_crawling_result.xlsx")
doc4 = pd.read_excel("c:\\naver_qna\\우울증_0301_0331_crawling_result.xlsx")
doc5 = pd.read_excel("c:\\naver_qna\\우울증_0401_0415_crawling_result.xlsx")
doc6 = pd.read_excel("c:\\naver_qna\\우울증_0416_0430_crawling_result.xlsx")

doc_list = ['doc1','doc2','doc3','doc4','doc5','doc6']

for i in doc_list:
    df = globals()[i]
    df.date = pd.to_datetime(df.date)

doc1 = doc1[(doc1.date >= '2022-01-01')&(doc1.date <= '2022-01-15')]
doc2 = doc2[(doc2.date >= '2022-01-16')&(doc2.date <= '2022-01-31')]
doc3 = doc3[(doc3.date >= '2022-02-01')&(doc3.date <= '2022-02-28')]
doc4 = doc4[(doc4.date >= '2022-03-01')&(doc4.date <= '2022-03-31')]
doc5 = doc5[(doc5.date >= '2022-04-01')&(doc5.date <= '2022-04-15')]
doc6 = doc6[(doc6.date >= '2022-04-16')&(doc6.date <= '2022-04-30')]

result = pd.concat([doc1,doc2,doc3,doc4,doc5,doc6]).reset_index()
result = result.iloc[:,1:]
result

# 전처리

## 컬럼병합

In [ ]:
df_qna = result.drop(['title', 'content'], axis = 1)
df_qna['data'] = result.title + ' ' + result.content
df_qna  # 총 24982건

## 결측치 제거

In [ ]:
df_qna.isnull().sum()  # data 컬럼에 결측치 3건
df_qna[df_qna['data'].isnull()] #결측데이터 확인

In [ ]:
df_qna.dropna(inplace = True)  # 결측치 제거

# 제거된 결측치 확인
df_qna.isnull().sum() 

## 중복데이터 제거

In [ ]:
# 중복된 데이터 제거
# 총 데이터 21536건
df_qna.drop_duplicates(inplace = True)
df_qna = df_qna.reset_index(drop = True)
df_qna

## 문자열 정제 및 토큰화

### 불용어 사전 로드

In [ ]:
# 불용어 파일 로드 
with open('c:\\naver_qna\\stopwords_korean.txt', encoding='utf8') as f:
    stopwords = f.read()
    stopwords = stopwords.split('\n')
        
stopwords = list(set(stopwords))

print(f'불용어는 총 {len(stopwords)}개 입니다. \
\n이 중 사용자가 임의로 추가한 불용어는 {len(stopwords)-675}개 입니다.')

### 토큰화

In [ ]:
# 불용어 제거
# Okt로 명사만 선택
# 한글, 영어, 숫자, 특수문자 제거
from konlpy.tag import Okt
from tqdm import tqdm
import re

okt = Okt()
data_sents = df_qna.data
data_corpus = []

'''
pattern1 : 날짜 단위 규칙
pattern2 : 나이 단위 규칙
pattern3 : 시간 단위 규칙
pattern4 : 학교 단위 규칙
'''

pattern1 = r"\d+년째|\d+년|\d+개월|\d+월|\d+일|\d+달|\d+달째|\d+일째"
pattern2 = r"\d+살|\d+세대|\d+대"
pattern3 = r"\d+시|\d+분|\d+시간"
pattern4 = r"고\d+|중\d+|초\d+|\d학년|\d학기"

for idx,sent in enumerate(tqdm(data_sents)):
    tokens = []
    try:
        new_sent = sent
        new_sent = re.sub(pattern1, "", new_sent)
        new_sent = re.sub(pattern2, "", new_sent)
        new_sent = re.sub(pattern3, "", new_sent)
        new_sent = re.sub(pattern4, "", new_sent)
        new_sent = re.sub(r"[^가-힣ㄱ-하-la-zA-Z ]","",new_sent)
        tokens = okt.nouns(new_sent)
        tokens = [word for word in tokens if word not in stopwords ]
        
    except:
        pass

    data_corpus.append(tokens)
    
df_qna['data_corpus'] = data_corpus

## 토큰화 후 결측 데이터 제거

In [ ]:
remove_idx = []
for idx, corpus in enumerate(df_qna.data_corpus):
    if len(corpus) == 0:
        print(idx, "|", df_qna.iloc[idx,-2],'\n')
        remove_idx.append(idx)

df_qna_fin= df_qna.drop(remove_idx).reset_index(drop = True)
df_qna_fin

## 단어수 데이터셋 생성
토큰화 후 전체 데이터셋의 단어 빈도수를 담은 데이터 프레임 생성

In [ ]:
from collections import Counter

cntlist = Counter()

for word in df_qna_fin.data_corpus:
    cntlist.update(word)
    
df_corpus = pd.DataFrame({'word' : cntlist.keys(),
                          'cnt' : cntlist.values()})

df_corpus.sort_values(by = 'cnt', ascending = False).head(20)

---
분석에 사용될 데이터 프레임은 아래와 같음.  

|데이터프레임명|컬럼명|설명|
|---|:---:|:---|
|**df_qna_fin**|date|게시날짜|
||user_name|사용자 정보|
||tag|태그|
||data|본문(작성제목+작성내용)|
||data_corpus|본문을 자연어처리하여 생성한 명사 리스트|  
|**df_corpus**|word|단어명|
||cnt|빈도수|

---

# 데이터 분석

## 게시글 유형 분석

### 게시글 태그 유형

In [ ]:
eda = df_qna_fin.copy()

# 태그1순위만 따로 추출
eda['tag_1'] = eda['tag'].apply(lambda x: x.split(',')[0])

#태그별 빈도수 추출
df_tag = eda.tag_1.value_counts().reset_index()
df_tag['pct'] = eda.tag_1.value_counts(normalize = True).reset_index().iloc[:,1] * 100
df_tag.pct = df_tag.pct.apply(lambda x:round(x, 2))
df_tag.columns = ['tag','cnt','pct']

# 상위 10개 + 그 외 태그들의 합
df_tag11 = df_tag.head(10)
df_tag11.loc[10] = ['그 외 1%이하', df_tag.cnt[11:].sum(), round(df_tag.pct[11:].sum(),2) ]
df_tag11

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드

cmap = plt.get_cmap('Spectral')
colors = [cmap(i) for i in np.linspace(0, 1, 10)] + ['lightgray']

fig, ax = plt.subplots(figsize=(20,10), subplot_kw=dict(aspect="equal"))

label = df_tag11['tag'] + ' ' + df_tag11.pct.apply(lambda x : str(x) + '%')
data = df_tag11['cnt']

wedges, texts = ax.pie(data, wedgeprops=dict(width=0.5), startangle=150, colors = colors, counterclock=False)

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(label[i], xy=(x, y), xytext=(1.05*np.sign(x), 1.1*y),  # 라벨 x,y위치 조절
                horizontalalignment=horizontalalignment, **kw)

plt.title('게시글 태그 분포', size = 15, pad = 20)
plt.show()

### 사용자 공개설정 여부

In [ ]:
# 사용자 공개여부 컬럼 생성
eda['username_1'] = eda.user_name.apply(lambda x: '비공개' if x == '비공개' else ('알 수 없음' if x == '정보가 없는 사용자' else '공개'))
user_x = eda.username_1.value_counts().index
user_y = eda.username_1.value_counts().values

plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드

# 그래프 그리기
plt.figure(figsize = (10,8))
x = plt.bar(user_x,user_y,
           width = 0.5)

plt.title('사용자 공개설정 여부', size = 15, pad = 20)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.ylim([0,18000])
plt.ylabel('(단위 : 건)', size = 12,labelpad = 10)
plt.grid(True,  alpha=0.3)

font = {'family': 'Arial',
      'color':  'black',
        'size': 15,
      'alpha': 0.7}

for idx, value in enumerate(user_y):
    text = '{:0,d}'.format(value)
    plt.text(idx-0.07, value+100, text, fontdict = font)

for i in list(['right','top','left']):
    plt.gca().spines[i].set_visible(False)
    
plt.show()

### 요일별 게시글 수

In [ ]:
# 요일별 빈도수를 담은 변수 생성
day_x = eda.date.dt.weekday.value_counts().sort_index().index
day_y = eda.date.dt.weekday.value_counts().sort_index().values

# 그래프 그리기
plt.figure(figsize = (10,5))
x = plt.bar(day_x, day_y,width = 0.6)

plt.title('요일별 게시글 수', size = 15, pad = 20)
plt.xticks(list(range(7)), labels = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일'], size = 12)
plt.yticks([0,1000,2000,3000,4000], size = 12)
plt.ylim([0,4000])
plt.ylabel('(단위 : 건)', size = 12,labelpad = 8)
plt.grid(True,  alpha=0.3)

font = {'family': 'Arial',
      'color':  'black',
        'size': 12,
      'alpha': 0.7}

for idx, value in enumerate(day_y):
    text = '{:0,d}'.format(value)
    plt.text(idx-0.18, value+100, text, fontdict = font)

for i in list(['right','top','left']):
    plt.gca().spines[i].set_visible(False)

plt.show()

### 월별 게시글 수

In [ ]:
# date에서 월(month) 데이터 추출하여 컬럼 생성
eda['date_month'] = eda.date.dt.month.apply(lambda x:str(x)+'월')

# 월별 데이터에서 빈도수를 담은 변수 생성
month_x = eda.date_month.value_counts().sort_index().index
month_y = eda.date_month.value_counts().sort_index().values

# 그래프 그리기
plt.figure(figsize = (8,5))
x = plt.bar(month_x, month_y, width = 0.5)

plt.title('월별 게시글 수', size = 15, pad = 20)

# plt.xticks(labels = month_x ,size = 12)
plt.yticks(np.arange(4000,6001,500), size = 12)
plt.ylim([4000,6000])
plt.ylabel('(단위 : 건)', size = 12,labelpad = 8)
plt.grid(True, alpha=0.3)

font = {'family': 'Arial',
      'color':  'black',
        'size': 12,
      'alpha': 0.7}

for idx, value in enumerate(month_y):
    text = '{:0,d}'.format(value)
    plt.text(idx-0.1, value+50, text, fontdict = font)        
    
for i in list(['right','top','left']):
    plt.gca().spines[i].set_visible(False)

plt.show()

### 월별 게시글 태그 상위 5개

In [ ]:
# 월별 게시글 태그 상위 5개 데이터 추출
tag_by_month = eda[['date_month','tag']].value_counts().reset_index()
tag_by_month.columns = ['month','tag','cnt']
tag_by_month = tag_by_month.sort_values(by = ['month', 'cnt'], ascending = [True, False]).groupby('month').head(5)

# ax2그래프의 변수 설정
tag_x = tag_by_month.groupby('month')['cnt'].sum().index
tag_y = tag_by_month.groupby('month')['cnt'].sum().values

# 그래프 그리기
plt.figure(figsize = (18,8))

custom_palette = sns.color_palette("Paired", 7)

ax1 = sns.barplot(x = 'month', y = 'cnt', hue = 'tag', data = tag_by_month, palette = custom_palette)
ax2 = plt.plot(tag_x, tag_y,'o--', color = 'black', label = '게시글 수') 

plt.legend(loc=(0.13, 1.03), ncol = 8, frameon = False)
plt.xticks(size = 12)
plt.yticks(size = 12)
plt.xlabel('month', size = 12)
plt.ylabel('cnt (단위 : 건)', size = 12)
plt.ylim([0,3000])

for container in ax1.containers:
    plt.bar_label(container, padding = 5, size = 12)

for x, y in zip(tag_x, tag_y):
    plt.text(x = x, y = y+70,
             s = "{:.0f}".format(y),
             size = 12)
    
plt.title('월별 게시글 태그 상위 5개', size = 15, pad = 40)
plt.grid(True, alpha=0.3)

for i in list(['right','top','left']):
    plt.gca().spines[i].set_visible(False)

plt.show()

### 게시글 글자수 

In [ ]:
print(f'''
게시글의 평균 길이는 {round(eda.data.apply(lambda x : len(x)).mean())}자 이다.
최소 길이는  {min(eda.data.apply(lambda x : len(x)))}자, 최대 길이는  {max(eda.data.apply(lambda x : len(x)))}자 이다.
''')

# 본문 글자수 계산한 컬럼 생성
eda['word_num'] = eda.data.apply(lambda x: len(x))
eda

In [ ]:
# 글자수 통계정보
eda[['word_num']].describe().apply(lambda x: x.astype(int))

#### 게시글별 글자수 분포

In [ ]:
plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드

# 그래프 그리기
plt.figure(figsize = (20,30))
sns.displot(eda.word_num, height = 5, aspect = 2)

plt.xticks(size = 12)
plt.yticks(size = 12)
plt.xlabel('단어수', size = 12)
plt.ylabel('Count', size = 12)
plt.xlim([0,max(eda.word_num)])
plt.title('게시글별 글자수 분포', size = 12, pad = 15)

plt.show()

#### 상위 1%를 제외한 게시글별 글자수 분포

In [ ]:
# 상위 1% 제외
cut_point = eda.word_num.quantile(0.99)
eda2 = eda[eda.word_num < cut_point]

# 그래프 그리기
plt.figure(figsize = (20,30))

plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드

sns.displot(eda2.word_num, height = 5, aspect = 2)
sns.set_style("whitegrid")

plt.xticks(size = 12)
plt.yticks(size = 12)
plt.xlabel('단어수', size = 12)
plt.ylabel('Count', size = 12)
plt.xlim([0,2100])

plt.title('(상위 1%를 제외한) 게시글별 글자수 분포', size = 12, pad  = 15)

plt.show()

### 단어 빈도수 워드클라우드

In [ ]:
# 단어 빈도수 데이터 프레임
wc_temp = pd.DataFrame(cntlist.most_common(20)[:10])
wc_temp2 = pd.DataFrame(cntlist.most_common(20)[10:])
df_wc = pd.concat([pd.DataFrame({'rank':list(range(1,11))}),wc_temp, 
                   pd.DataFrame({'rank':list(range(11,21))}),wc_temp2], axis = 1)
df_wc.columns = ['rank','word', 'cnt(단위 : 건)']*2
df_wc

In [ ]:
# 워드클라우드
def word_cloud(wc_list,colormap) :
    from wordcloud import WordCloud
    import matplotlib.pyplot as plt
    from collections import Counter
    from konlpy.tag import Okt
    from PIL import Image
    import numpy as np
    
    img = Image.open('c:\\naver_qna\\head.png')  #워드클라우드 배경이미지
    img_array = np.array(img)

    wc = WordCloud(background_color ='white',colormap = colormap,
                   width = 700, height = 700, font_path='H2GTRE',
                   max_font_size=300, max_words=150, mask = img_array)

    gen = wc.generate_from_frequencies(wc_list)

    plt.figure(figsize = (6,6))
    plt.imshow(gen)
    plt.axis('off')
    plt.show()

#     wc.to_file('c:\\naver_qna\\qna_wordcloud_con_corpus.jpg')

word_cloud(cntlist, 'gist_earth')

## 군집분석

### Word2Vec
gensim의 패키지 Word2Vec을 사용하여 벡터화. vector_size = 100, 단어 등장횟수 10번 이상, CBoW로 진행

In [ ]:
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from matplotlib import font_manager as fm
from matplotlib import rc

word_corpus = df_qna_fin.data_corpus.to_list()
model = Word2Vec(word_corpus, vector_size = 100, window = 5, min_count = 20, sg = 0, seed = 100)

# # 단어 유사도 분석
# model.wv.most_similar('우울', topn=20)

### 차원축소

In [ ]:
tsne = TSNE(n_components = 2, random_state = 100)

vocab = model.wv.key_to_index.keys()
similarity = model.wv[vocab]
print(similarity.shape)

transform_similarity = tsne.fit_transform(similarity) # 2차원으로 변환됨
df_tsne = pd.DataFrame(transform_similarity, index = vocab, columns = ['x','y']).reset_index()
df_tsne.columns = ['word','x','y']
df_tsne[:10]

### k-means 

In [ ]:
# elbow method
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

sse = []
for i in range(1,11):
    km = KMeans(n_clusters = i, init = 'k-means++', random_state = 0)
    km.fit(df_tsne.iloc[:,1:]) #x, y 좌표만 넣음
    sse.append(km.inertia_)

plt.figure(figsize = (6,5))
plt.plot(range(1,11), sse, marker = 'o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.title('KMEANS ElBOW METHOD')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
from collections import Counter

# 군집분석
kmeans = KMeans(n_clusters = 3, random_state = 100)
predict = kmeans.fit_predict(df_tsne.iloc[:,1:])

# 데이터에 predict 컬럼 추가
colordict = {0:'cluster1', 1:'cluster2', 2:'cluster3'}
df_tsne['predict'] = list(map(lambda x: colordict[x], predict))

In [ ]:
# 단어수 추가
for word in tqdm(df_tsne.word):
    df_tsne.loc[df_tsne.word == word, 'cnt'] = df_corpus[df_corpus.word == word]['cnt'].values[0]
    
df_tsne

### 그래프 그리기1 - seaborn

In [ ]:
# 군집 시각화 함수 생성
def Draw_Cluster(dataframe,hue,legend):
    plt.rc('font',family = 'Malgun Gothic')  # 한글 안깨지게 하는 코드
    plt.rcParams['axes.unicode_minus'] = False # 특수문자 안깨지는 코드
    
    sns.lmplot('x','y', 
               data = dataframe, 
               fit_reg = False, 
               size = 6, 
               hue = hue, 
               legend = True, palette = 'deep')
#     plt.legend(legend)
    plt.show()

def Find_ClusterCenter(dataframe,hue,palette,legend):
    sns.lmplot('x','y', data = dataframe, 
               fit_reg = False, size = 6, 
               hue = 'predict', legend = False,
          palette = palette)
    
    plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], 
                c = 'red', marker = '*', s = 50, label = '클러스터 중심')

    plt.legend(legend)
    plt.show()
    
palette = ['lightsteelblue', 'bisque','darkseagreen','lightpink']
legend = ['cluster1', 'cluster2', 'cluster3']

# 함수 호출
Draw_Cluster(df_tsne, 'predict',legend)
# Find_ClusterCenter(df_tsne, 'predict', palette, legend)

In [ ]:
# 군집별로 시각화
def graph_Center(dataframe, pred_num, title, color):
    palette = ['lightgray'] * 4
    palette[pred_num] = color
    
    sns.lmplot('x','y', data = dataframe, 
               fit_reg = False, size = 6, 
               hue = 'predict', legend = False,
               palette = palette)
    plt.scatter(kmeans.cluster_centers_[pred_num,0], 
                kmeans.cluster_centers_[pred_num,1], 
                c = 'red', marker = '*', s = 50, label = '클러스터 중심')

    plt.title(title, size = 20)
    plt.show()
    
graph_Center(df_tsne, 0, legend[0], palette[0])
graph_Center(df_tsne, 1, legend[1], palette[1])
graph_Center(df_tsne, 2, legend[2], palette[2])

In [ ]:
# 군집별 단어 등장횟수에 따른 순위
def sort_cnt_cluster(dataframe, pred_num):
    df_temp = dataframe[dataframe.predict == pred_num]
    df_temp = df_temp[['word','cnt']].sort_values('cnt', ascending = False).reset_index(drop = True)
    
    return df_temp
    
cnt1 = sort_cnt_cluster(df_tsne, 'cluster1')
cnt2 = sort_cnt_cluster(df_tsne, 'cluster2')
cnt3 = sort_cnt_cluster(df_tsne, 'cluster3')

df_sort_cnt = pd.DataFrame(pd.concat([cnt1,cnt2,cnt3], axis = 1))

column = pd.MultiIndex.from_product([['cluster1','cluster2','cluster3'],['word','cnt']], names = ['cluster','column'])
df_sort_cnt.columns = column

df_sort_cnt.head(20)

### 그래프 그리기2 - Plotly

In [ ]:
import chart_studio
# username, api_key 입력
chart_studio.tools.set_credentials_file(username = , api_key= )

import plotly
import plotly.graph_objs as go
import plotly.express as px
import chart_studio.plotly as py

fig = px.scatter(df_tsne,
                 x='x',
                 y='y',
                 color='predict', 
                 title = '군집분석', 
                 size = 'cnt', size_max = 80, hover_name="word"
                )

fig.update_traces(textfont_size=15, textposition='top center')

# py.iplot(fig)
py.plot(fig, filename = 'cluster_fin', auto_open=True)
# fig.show()

## 연관분석 및 네트워크 신경망

In [ ]:
## 연관분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit_transform(word_corpus)
df = pd.DataFrame(te_ary, columns = te.columns_)
df

In [ ]:
frequent_itemsets = apriori(df, min_support = 0.04, use_colnames = True)
frequent_itemsets

In [ ]:
# 단어2개 쌍만 추출
apri = association_rules(frequent_itemsets, metric="lift", min_threshold = 1)
apri = apri[(apri.antecedents.apply(lambda x: len(x) == 1))&(apri.consequents.apply(lambda x: len(x) == 1))]
apri = apri.explode('antecedents').explode('consequents')
apri

### 네트워크 신경망 그리기

In [ ]:
import networkx as nx
from pyvis.network import Network

G = nx.from_pandas_edgelist(apri,
                          source = "antecedents",
                           target = "consequents")

def draw_network(network):
    net = Network("600px", "1000px", notebook=False)
    palette = ['steelblue', 'darkorange','darkgreen']
    
    # node_size
    for node in network.nodes():
        network.nodes[node]['size'] = int(df_corpus['cnt'][df_corpus.word == node]) / 500
        
        colordict = {'cluster1':'orangered', 'cluster2':'lightseagreen', 'cluster3':'slateblue'}
        color = df_tsne[['predict']][df_tsne.word == node].iloc[0,0]
        network.nodes[node]['color'] = colordict[color]

    net.from_nx(network)
    net.show('network.html')

draw_network(G)